In [1]:
from matplotlib import pyplot as plt
# from astropy.utils.data import download_file
# from astropy.time import Time, TimeDelta
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.time import Time, TimeDelta
# import xml.etree.ElementTree as ET, gzip, io

from dateutil.parser import parse as date_parse
import numpy as np
import pandas as pd

from collections import namedtuple
from glob import glob
from astropy.io import fits
from astropy.table import Table

from tqdm import tqdm_notebook

from piaa import exoplanets
import exodata

In [3]:
exoplanet = exoplanets.Exoplanet('Hd189733')

In [2]:
!gsutil ls gs://panoptes-survey/PAN001/**/**/2018*/*.fz > /var/panoptes/pan001.txt

In [3]:
!cat /var/panoptes/pan001.txt | grep -v 'pointing' | cut -d '/' -f 4-7 | sort | uniq > /var/panoptes/pan001_seq.txt

In [7]:
# Load our list of stored images
txt_fn = '/var/panoptes/pan001.txt'

fits_fn = !cat $txt_fn | grep -v 'pointing' | grep -v 'ee04d1'

In [8]:
len(fits_fn)

15897

In [9]:
fits_fn[0:4]

['gs://panoptes-survey/PAN001/55Cnc/14d3bd/20180216T110623/20180216T110816.fits.fz',
 'gs://panoptes-survey/PAN001/55Cnc/14d3bd/20180216T110623/20180216T111125.fits.fz',
 'gs://panoptes-survey/PAN001/55Cnc/14d3bd/20180216T110623/20180216T111430.fits.fz',
 'gs://panoptes-survey/PAN001/55Cnc/14d3bd/20180216T110623/20180216T111740.fits.fz']

In [ ]:
transits = dict()
errors = dict()

for fn in tqdm_notebook(fits_fn[::5]):
    if 'pointing' in fn:
        continue 
    
    try:
        _, _, _, unit_id, field, cam_id, seq_id, img_id = fn.split('/')
    except ValueError:
        continue 
        
    if seq_id in transits:
        continue
        
    # Get the time from the image
    img_id = img_id.split('.')[0]
    try:
        t0 = Time(date_parse(img_id), format='datetime')
    except Exception as e:
        if str(e) not in errors:
            print(img_id, e)
            errors[str(e)] = True
        continue
        
    # Get exoplanet info
    try:
        planet = exoplanets.Exoplanet(field)
    except KeyError as e:
        if str(e) not in errors:
            print(e)
            errors[str(e)] = True        
        continue
        
    try:
        if planet.transit_duration.value == 0:
            k = '{}_duration'.format(planet.name)
            if k not in errors:
                print("No duration for", planet.name)
                errors[k] = True
            continue

        if planet.transit_midpoint.value == 0:
            k = '{}_midpoint'.format(planet.name)
            if k not in errors:
                print("No midpoint for", planet.name)
                errors[k] = True
            continue        

        if planet.period.value == 0:
            k = '{}_period'.format(planet.name)
            if k not in errors:
                print("No period for", planet.name)
                errors[k] = True
            continue                
    except Exception as e:
        if str(e) not in errors:
            print(e)
            errors[str(e)] = True                
    
    # Determine if in transit
    try:
        in_t = planet.in_transit(t0, with_times=True)
    except Exception as e:
        if str(e) not in errors:
            print(e)
            errors[str(e)] = True        
        continue    

    if in_t[0]:
        transits[seq_id] = {'transit': in_t[1], 'planet': planet}


The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()
'No matches found for key Epic211089792'
list index out of range
'No matches found for key Kic8462852'
'No matches found for key M5'


In [ ]:
# Three subplots sharing both x/y axes

f, axes = plt.subplots(len(transits), sharey=True)
f.set_size_inches(12, 2 * len(transits))

for i, seq_id in enumerate(transits.keys()):

    try:
        ax = axes[i]
    except Exception:
        ax = axes
    
    t = transits[seq_id]['transit']
    planet = transits[seq_id]['planet']
    image_times = sorted(set(([x.split('/')[7].split('.')[0] for x in fits_fn if seq_id in x])))
                                
    phases = {planet.get_phase(Time(date_parse(t))):Time(date_parse(t)) for t in image_times}
    
    ax.scatter(list(phases.keys()), [1 for x in range(len(phases))], color='b', marker='.', label='Observation')
    
#     print(t)
#     print(planet.get_phase(t.ingress), planet.get_phase(t.egress))
#     print()
    ax.axvline(planet.get_phase(t.ingress), 0, color='r', ls='--', alpha=0.5, label='ingress')
    ax.axvline(planet.get_phase(t.egress), 0, color='r', ls='--', alpha=0.5, label='egress')
        
    ax.legend()
        
    try:
        ax.set_title("{} {} {} {}".format(planet.info['pl_hostname'], seq_id, planet.star_mag, planet.transit_depth))
    except Exception:
        pass
    ax.set_xlim(-0.5, 0.5)
    ax.set_ylim(0.5, 1.5)
    ax.get_yaxis().set_ticks([])
    
    f.subplots_adjust(hspace=0)
    
f.savefig(txt_fn.replace('.txt', '_transits.png'))

In [13]:
list(transits.keys())

['20180202T061353',
 '20180412T063417',
 '20180310T123739',
 '20180330T111258',
 '20180614T093015',
 '20180813T090404',
 '20180605T092351',
 '20180626T093238',
 '20180131T120232',
 '20180517T091558',
 '20180521T092247',
 '20180607T090809',
 '20180611T064835',
 '20180624T063045',
 '20180215T131704',
 '20180217T073053',
 '20180412T094809',
 '20180419T122738',
 '20180426T091237',
 '20180515T061520',
 '20180605T062206',
 '20180123T073651',
 '20180131T085025',
 '20180307T081001',
 '20180411T081824',
 '20180411T084040',
 '20180609T095151',
 '20180221T101626']